# Initialization

The GPU can be initialized automatically by importing the `autoinit` module.

In [1]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
from skcuda import cublas
import sys
from time import time

/home/gjb/miniconda3/envs/cuda/lib/python3.9/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


# cuBLAS

The BLAS library supports

* level 1: vector operations,
* level 2: matrix-vector operations,
* level 3: matrix-matrix operations.

scikit-cuda provides bindings to NVIDIA's cuBLAS library.  It is interoperable with PyCUDA's `gpuarray` representation.

## Creating a context

To perform cuBLAS computations, a context has to be instantiated.  The handler will be passed to all the wrapper functions that provide the BLAS operations.

In [3]:
cublas_h = cublas.cublasCreate()

When the cuBLAS computations are done, the context should be destroyed.

## Level 1: saxpy

The saxpy operation scales a vector and adds a second vector to it, i.e.,
$$
    \vec{y} = \alpha \vec{x} + \vec{y}
$$
Note that the original vector $\vec{y}$ is overwritten.

We create the data to experiments with.

In [4]:
size = 1_000_000
α = np.float32(3.1)
x = np.float32(np.random.uniform(size=(size, )))
y = np.float32(np.random.uniform(size=(size, )))

In [5]:
dev_x = gpuarray.to_gpu(x)
dev_y = gpuarray.to_gpu(y)

The saxpy operation is called on the context handle, $\alpha$ and both vectors are passed to it.  Note that `cublasSaxpy` expects the addresses of the data on the device.  The latter can be obtained from the GPU array using the `gpudata` attribute.  We also need to provide a stride for both the $x$ and $y$ vectors, which is 1 in our caes.

In [6]:
cublas.cublasSaxpy(cublas_h, dev_x.size, α, dev_x.gpudata, 1, dev_y.gpudata, 1)

In [7]:
y_result = dev_y.get()

In [8]:
np.allclose(y_result, α*x + y, rtol=1.0e-5)

True

## Destroying the context

Once done, the context should be destroyed.

In [9]:
cublas.cublasDestroy(cublas_h)

## Level 3: sgemm

In [10]:
cublas_h = cublas.cublasCreate()

The sgemm operations multiplies and scales two matrices, and adds the result to a third matrix, i.e.,
$$
    C = \alpha A \cdot B + \beta C
$$
We initialize the matrices and constants.

In [11]:
m, k, n = 2_000, 3_000, 4_000
A = np.float32(np.random.uniform(size=(m, k)))
B = np.float32(np.random.uniform(size=(k, n)))
C = np.float32(np.random.uniform(size=(m, n)))
α = np.float32(3.1)
β = np.float32(0.7)

Note that the BLAS assumes that data is stored columnwise.  However, numpy by default stores matrices rowwise, so we transpose the matrices before transferring them to the device memory.  Also note that the data has to be copied after the transpose.

In [12]:
dev_A = gpuarray.to_gpu(A.T.copy())
dev_B = gpuarray.to_gpu(B.T.copy())
dev_C = gpuarray.to_gpu(C.T.copy())

For an sgemm operation, the matrices $A$ and $B$ can be transposed, but we do not want to do that here, so `transA` and `transB` are set to `'N'`.  We also need to specify the leading dimension for each of the matrices, i.e., `lda`, `ldb`, `ldc`.

In [13]:
transA = cublas._CUBLAS_OP['N']
transB = cublas._CUBLAS_OP['N']
lda, ldb, ldc = m, k, m

Now the sgemm operation can be performed.

In [14]:
cublas.cublasSgemm(cublas_h, transA, transB, m, n, k, α, dev_A.gpudata, lda, dev_B.gpudata, ldb, β, dev_C.gpudata, ldc)

The result is the same as on the host.

In [15]:
np.allclose(dev_C.get().T, α*np.dot(A, B) + β*C)

True

In [16]:
cublas.cublasDestroy(cublas_h)

## Matrix power

We want to compute the matrix power $A^p = A \cdot \cdots \cdot A$ using the sgemm operation.  Since we have
$$
    C = \alpha A \cdot B + \beta C
$$
we can compute the power of $A$ by iterating for $\alpha = 1$ and $\beta = 0$ and
$$
    B_{i+1} = A \cdot B_i
$$
setting $B_1 = A$.

In [24]:
p = 1_000
m, k, n = 4_000, 4_000, 4_000
A = np.float32(np.random.uniform(0.0, 1.0, size=(m, k)))
α, β = np.float32(1.0), np.float32(0.0)

In [25]:
transA = cublas._CUBLAS_OP['N']
transB = cublas._CUBLAS_OP['N']
lda, ldb, ldc = m, k,m

In [26]:
%%time
cublas_h = cublas.cublasCreate()
dev_A = gpuarray.to_gpu(A.copy().T)
dev_B = gpuarray.to_gpu(A.copy().T)
dev_C = gpuarray.empty_like(dev_A)
start_time = time()
for _ in range(p - 1):
    cublas.cublasSgemm(cublas_h, transA, transB, m, n, k, α, dev_A.gpudata, lda, dev_B.gpudata, ldb, β, dev_C.gpudata, ldc)
    dev_B = dev_C
end_time = time()
result = dev_B.get().copy().T
cublas.cublasDestroy(cublas_h)

CPU times: user 17.8 s, sys: 19.1 ms, total: 17.8 s
Wall time: 17.8 s


In [27]:
end_time - start_time

0.03950810432434082

In [28]:
%%time
host_result = np.linalg.matrix_power(A, p)

CPU times: user 40.6 s, sys: 7.34 s, total: 47.9 s
Wall time: 3 s


In [29]:
np.allclose(result, host_result)

True

### Benchmarking

In [2]:
p_values = [10, 100, 1_000, 10_000]
size_values = [100, 200, 500, 1_000, 2_000, 5_000]
α, β = np.float32(1.0), np.float32(0.0)
transA = cublas._CUBLAS_OP['N']
transB = cublas._CUBLAS_OP['N']
cublas_h = cublas.cublasCreate()
dev_time = np.empty((len(p_values), len(size_values)))
dev_comp_time = np.empty((len(p_values), len(size_values)))
host_comp_time = np.empty((len(p_values), len(size_values)))
for i_p, p in enumerate(p_values):
    for i_size, size in enumerate(size_values):
        print(f'starting p = {p}, size = {size}')
        m, k, n = size, size, size
        A = np.float32(np.random.uniform(0.0, 1.0, size=(m, k)))
        lda, ldb, ldc = m, k, m
        dev_start_time = time()
        dev_A = gpuarray.to_gpu(A.copy().T)
        dev_B = gpuarray.to_gpu(A.copy().T)
        dev_C = gpuarray.empty_like(dev_A)
        dev_start_comp_time = time()
        for _ in range(p - 1):
            cublas.cublasSgemm(cublas_h, transA, transB, m, n, k, α, dev_A.gpudata, lda, dev_B.gpudata, ldb, β, dev_C.gpudata, ldc)
            dev_B = dev_C
        end_time = time()
        result = dev_B.get().copy().T
        dev_end_comp_time = time()
        dev_end_time = time()
        dev_time[i_p, i_size] = dev_end_time - dev_start_time
        dev_comp_time[i_p, i_size] = dev_end_comp_time - dev_start_comp_time
        host_start_comp_time = time()
        host_result = np.linalg.matrix_power(A, p)
        host_end_comp_time = time()
        host_comp_time[i_p, i_size] = host_end_comp_time - host_start_comp_time
        if not np.allclose(result, host_result):
            print(f'not equal for {p}, {size}', file=sys.stderr)
cublas.cublasDestroy(cublas_h)

starting p = 10, size = 100
starting p = 10, size = 200
starting p = 10, size = 500
starting p = 10, size = 1000
starting p = 10, size = 2000


not equal for 10, 1000
not equal for 10, 2000


starting p = 10, size = 5000


not equal for 10, 5000
/home/gjb/miniconda3/envs/cuda/lib/python3.9/site-packages/numpy/linalg/linalg.py:661: RuntimeWarning: overflow encountered in matmul
  z = a if z is None else fmatmul(z, z)
/home/gjb/miniconda3/envs/cuda/lib/python3.9/site-packages/numpy/linalg/linalg.py:664: RuntimeWarning: invalid value encountered in matmul
  result = z if result is None else fmatmul(result, z)
/home/gjb/miniconda3/envs/cuda/lib/python3.9/site-packages/numpy/linalg/linalg.py:661: RuntimeWarning: invalid value encountered in matmul
  z = a if z is None else fmatmul(z, z)


starting p = 100, size = 100
starting p = 100, size = 200
starting p = 100, size = 500
starting p = 100, size = 1000
starting p = 100, size = 2000
starting p = 100, size = 5000
starting p = 1000, size = 100
starting p = 1000, size = 200
starting p = 1000, size = 500
starting p = 1000, size = 1000
starting p = 1000, size = 2000
starting p = 1000, size = 5000
starting p = 10000, size = 100
starting p = 10000, size = 200
starting p = 10000, size = 500
starting p = 10000, size = 1000
starting p = 10000, size = 2000
starting p = 10000, size = 5000


In [3]:
host_comp_time

array([[1.07288361e-02, 1.30517483e-02, 5.44724464e-02, 3.40552330e-02,
        1.51504278e-01, 1.60136175e+00],
       [8.78953934e-03, 2.12025642e-03, 2.43089199e-02, 1.26446247e-01,
        4.27002430e-01, 3.55468321e+00],
       [4.15921211e-03, 4.21645641e-02, 1.37926102e-01, 3.03728342e-01,
        9.36742544e-01, 5.39860249e+00],
       [1.43671036e-03, 2.12764740e-03, 1.63109303e-02, 1.11539602e-01,
        6.37586355e-01, 6.56140661e+00]])

In [4]:
dev_comp_time

array([[6.03675842e-04, 8.99076462e-04, 1.65820122e-03, 6.42848015e-03,
        3.89833450e-02, 5.29721737e-01],
       [1.94239616e-03, 2.11763382e-03, 6.47735596e-03, 4.05559540e-02,
        2.73972034e-01, 3.61101055e+00],
       [1.74765587e-02, 7.97739029e-02, 5.82485199e-02, 3.34474325e-01,
        2.29343367e+00, 3.48084452e+01],
       [1.70212984e-01, 1.55580997e-01, 4.42224264e-01, 3.10729861e+00,
        2.33812053e+01, 3.53068779e+02]])

In [5]:
dev_time

array([[4.32896614e-03, 2.20060349e-03, 7.27701187e-03, 1.08642578e-02,
        5.36551476e-02, 5.99342346e-01],
       [6.13522530e-03, 2.62570381e-03, 8.02016258e-03, 4.95145321e-02,
        2.92020559e-01, 3.67820096e+00],
       [2.10342407e-02, 8.08649063e-02, 6.34791851e-02, 3.38699102e-01,
        2.31347179e+00, 3.48757985e+01],
       [1.74494505e-01, 1.62458181e-01, 4.45505857e-01, 3.11164284e+00,
        2.33960242e+01, 3.53136671e+02]])